In [45]:
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

In [32]:
log = pd.read_csv('log.csv')
log.head()

,date,user,event_type,parameter
0,2020-04-01_00:04:03,976455,3,NaN
1,2020-04-01_00:01:27,456430,3,text
2,2020-04-01_00:06:32,566998,1,unknown
3,2020-04-01_00:08:28,927997,3,NaN
4,2020-04-01_00:08:53,237940,4,unknown


In [36]:
log['date'] = pd.to_datetime(log['date'], format='%Y-%m-%d_%H:%M:%S')
log.head()

,date,user,event_type,parameter
0,2020-04-01 00:04:03,976455,3,NaN
1,2020-04-01 00:01:27,456430,3,text
2,2020-04-01 00:06:32,566998,1,unknown
3,2020-04-01 00:08:28,927997,3,NaN
4,2020-04-01 00:08:53,237940,4,unknown


## Задание 1

Посчитайте количество сессий, которые начались 2020-04-19. Сессия заканчивается, если пользователь не совершал действий в течение 30 минут или более.

In [40]:
log.sort_values(['user', 'date'], inplace=True)
log.head()

,date,user,event_type,parameter
5692,2020-04-02 10:39:10,100081,2,text
7929,2020-04-02 15:37:13,100081,2,NaN
16401,2020-04-03 15:36:01,100081,4,unknown
24724,2020-04-05 07:26:55,100081,2,unknown
27129,2020-04-05 13:05:31,100081,4,music


In [43]:
log['delta'] = log['date'] - log['date'].shift(periods=1)
log.head()

,date,user,event_type,parameter,delta
5692,2020-04-02 10:39:10,100081,2,text,NaT
7929,2020-04-02 15:37:13,100081,2,NaN,0 days 04:58:03
16401,2020-04-03 15:36:01,100081,4,unknown,0 days 23:58:48
24724,2020-04-05 07:26:55,100081,2,unknown,1 days 15:50:54
27129,2020-04-05 13:05:31,100081,4,music,0 days 05:38:36


In [46]:
mask = log['user'] != log['user'].shift(periods=1)
log['delta'][mask] = pd.NaT
log.head()

,date,user,event_type,parameter,delta
5692,2020-04-02 10:39:10,100081,2,text,NaT
7929,2020-04-02 15:37:13,100081,2,NaN,0 days 04:58:03
16401,2020-04-03 15:36:01,100081,4,unknown,0 days 23:58:48
24724,2020-04-05 07:26:55,100081,2,unknown,1 days 15:50:54
27129,2020-04-05 13:05:31,100081,4,music,0 days 05:38:36


In [58]:
log['day'] = log['date'].astype('datetime64[D]')
log.head()

,date,user,event_type,parameter,delta,day
5692,2020-04-02 10:39:10,100081,2,text,NaT,2020-04-02
7929,2020-04-02 15:37:13,100081,2,NaN,0 days 04:58:03,2020-04-02
16401,2020-04-03 15:36:01,100081,4,unknown,0 days 23:58:48,2020-04-03
24724,2020-04-05 07:26:55,100081,2,unknown,1 days 15:50:54,2020-04-05
27129,2020-04-05 13:05:31,100081,4,music,0 days 05:38:36,2020-04-05


In [91]:
mask = log['delta'] >= pd.Timedelta(30, 'm')
len(log[mask].query('day=="2020-04-19"'))

6251

## Задание 2

Найдите день, когда число уникальных пользователей, просмотревших видео, было максимальным. В ответе укажите этот максимум.

Будем считать, что пользователь посмотрел видео, если у него было событие с типом 2 и параметром video.

In [75]:
ans = log.query('event_type==2 & parameter=="video"')
ans.head()

,date,user,event_type,parameter,delta,day
135618,2020-04-21 14:19:35,100081,2,video,0 days 00:30:00,2020-04-21
112539,2020-04-17 19:28:16,100112,2,video,0 days 07:37:26,2020-04-17
27423,2020-04-05 13:42:06,100443,2,video,1 days 16:08:08,2020-04-05
13544,2020-04-03 10:03:23,100469,2,video,0 days 13:44:27,2020-04-03
109178,2020-04-17 12:29:19,100469,2,video,3 days 19:22:18,2020-04-17


In [76]:
ans = ans.groupby(by='day').agg({'user':'nunique'})
ans.head()

,user
day,
2020-04-01,121
2020-04-02,303
2020-04-03,343
2020-04-04,137
2020-04-05,342


In [77]:
ans['user'].max()

391

## Задание 3 

Найдите пятиминутный интервал (time; time + 5 minutes), в течение которого произошло больше всего событий. 

В ответе укажите время начала интервала в формате YYYY-MM-DD_hh:mm:ss. Если таких интервалов несколько, укажите начало наиболее позднего интервала. Обратите внимание, что началом интервала может быть произвольный момент времени.

In [78]:
log.sort_values(by='date', inplace=True)
log.head()

,date,user,event_type,parameter,delta,day
1,2020-04-01 00:01:27,456430,3,text,NaT,2020-04-01
0,2020-04-01 00:04:03,976455,3,NaN,NaT,2020-04-01
2,2020-04-01 00:06:32,566998,1,unknown,NaT,2020-04-01
3,2020-04-01 00:08:28,927997,3,NaN,NaT,2020-04-01
4,2020-04-01 00:08:53,237940,4,unknown,NaT,2020-04-01


In [85]:
dates = log['date']
dates.head()

1   2020-04-01 00:01:27
0   2020-04-01 00:04:03
2   2020-04-01 00:06:32
3   2020-04-01 00:08:28
4   2020-04-01 00:08:53
Name: date, dtype: datetime64[ns]

In [86]:
delta = pd.Timedelta(5, 'm')
delta

Timedelta('0 days 00:05:00')

In [88]:
max_events = 0
start_date = None

i = 0
j = 0

while j < len(dates):
    if j < len(dates) and dates[j] - dates[i] < delta:
        j += 1
    else:
        i += 1
    events = j - i # количество событий в этом интервале
    if events >= max_events:
        max_events = events
        start_date = dates[i]

start_date

Timestamp('2020-04-09 13:06:42')